# Profiling and Timing Code

## showing only execution time in wandb

The main takeaway from Profiling weights and biases is that we sometimes need to print only the time module. If we want to print !only! the time module, we can do the following trick **exactly** in the given sequence:

```python
%%timeit -n 100
%%capture outputwandblog
```

and supress the warnings using the 

```python
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
```

- `%%timeit -n 100 -o` captures the output, which can be accesed as workaround in the next cell by the most recent: output 
```python
res = _
res
``` 
which is however not giving the most accurate "last" result (so we gonna try to avoid using that)

- to show the output of the capture magic use 
```python 
outputwandblog.show() 
``` 


## IPython magic commands

IPython magic commands:

- ``%time``: Time the execution of a single statement
- ``%timeit``: Time repeated execution of a single statement for more accuracy
- ``%prun``: Run code with the profiler
- ``%lprun``: Run code with the line-by-line profiler
- ``%memit``: Measure the memory use of a single statement
- ``%mprun``: Run code with the line-by-line memory profiler

## Timing Code Snippets: ``%timeit`` and ``%time``

We saw the ``%timeit`` line-magic and ``%%timeit`` cell-magic in the introduction to magic functions in [IPython Magic Commands](01.03-Magic-Commands.ipynb); it can be used to time the repeated execution of snippets of code:

In [1]:
%timeit sum(range(100))

100000 loops, best of 3: 1.54 µs per loop


Note that because this operation is so fast, ``%timeit`` automatically does a large number of repetitions.
For slower commands, ``%timeit`` will automatically adjust and perform fewer repetitions:

In [2]:
%%timeit
total = 0
for i in range(1000):
    for j in range(1000):
        total += i * (-1) ** j

1 loops, best of 3: 407 ms per loop


Sometimes repeating an operation is not the best option.
For example, if we have a list that we'd like to sort, we might be misled by a repeated operation.
Sorting a pre-sorted list is much faster than sorting an unsorted list, so the repetition will skew the result:

In [3]:
import random
L = [random.random() for i in range(100000)]
%timeit L.sort()

100 loops, best of 3: 1.9 ms per loop


For this, the ``%time`` magic function may be a better choice. It also is a good choice for longer-running commands, when short, system-related delays are unlikely to affect the result.
Let's time the sorting of an unsorted and a presorted list:

In [4]:
import random
L = [random.random() for i in range(100000)]
print("sorting an unsorted list:")
%time L.sort()

sorting an unsorted list:
CPU times: user 40.6 ms, sys: 896 µs, total: 41.5 ms
Wall time: 41.5 ms


In [5]:
print("sorting an already sorted list:")
%time L.sort()

sorting an already sorted list:
CPU times: user 8.18 ms, sys: 10 µs, total: 8.19 ms
Wall time: 8.24 ms


Notice how much faster the presorted list is to sort, but notice also how much longer the timing takes with ``%time`` versus ``%timeit``, even for the presorted list!
This is a result of the fact that ``%timeit`` does some clever things under the hood to prevent system calls from interfering with the timing.
For example, it prevents cleanup of unused Python objects (known as *garbage collection*) which might otherwise affect the timing.
For this reason, ``%timeit`` results are usually noticeably faster than ``%time`` results.

For ``%time`` as with ``%timeit``, using the double-percent-sign cell magic syntax allows timing of multiline scripts:

In [6]:
%%time
total = 0
for i in range(1000):
    for j in range(1000):
        total += i * (-1) ** j

CPU times: user 504 ms, sys: 979 µs, total: 505 ms
Wall time: 505 ms


For more information on ``%time`` and ``%timeit``, as well as their available options, use the IPython help functionality (i.e., type ``%time?`` at the IPython prompt).

## Profiling Full Scripts: ``%prun`` (really useful!)

A program is made of many single statements, and sometimes timing these statements in context is more important than timing them on their own.
Python contains a built-in code profiler (which you can read about in the Python documentation), but IPython offers a much more convenient way to use this profiler, in the form of the magic function ``%prun``.

By way of example, we'll define a simple function that does some calculations:

In [2]:
def sum_of_lists(N):
    total = 0
    for i in range(5):
        L = [j ^ (j >> i) for j in range(N)]
        total += sum(L)
    return total

Now we can call ``%prun`` with a function call to see the profiled results:

In [8]:
%prun sum_of_lists(1000000)

In the notebook, the output is printed to the pager, and looks something like this:

```
14 function calls in 0.714 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        5    0.599    0.120    0.599    0.120 <ipython-input-19>:4(<listcomp>)
        5    0.064    0.013    0.064    0.013 {built-in method sum}
        1    0.036    0.036    0.699    0.699 <ipython-input-19>:1(sum_of_lists)
        1    0.014    0.014    0.714    0.714 <string>:1(<module>)
        1    0.000    0.000    0.714    0.714 {built-in method exec}
```

The result is a table that indicates, in order of total time on each function call, where the execution is spending the most time. In this case, the bulk of execution time is in the list comprehension inside ``sum_of_lists``.
From here, we could start thinking about what changes we might make to improve the performance in the algorithm.

For more information on ``%prun``, as well as its available options, use the IPython help functionality (i.e., type ``%prun?`` at the IPython prompt).

## Line-By-Line Profiling with ``%lprun``

The function-by-function profiling of ``%prun`` is useful, but sometimes it's more convenient to have a line-by-line profile report.
This is not built into Python or IPython, but there is a ``line_profiler`` package available for installation that can do this.
Start by using Python's packaging tool, ``pip``, to install the ``line_profiler`` package:

```
$ pip install line_profiler
```

Next, you can use IPython to load the ``line_profiler`` IPython extension, offered as part of this package:

In [1]:
%load_ext line_profiler

GitHub repository: https://github.com/rkern/line_profiler

Now the ``%lprun`` command will do a line-by-line profiling of any function–in this case, we need to tell it explicitly which functions we're interested in profiling:

In [2]:
%lprun -f sum_of_lists sum_of_lists(5000)

UsageError: Could not find function 'sum_of_lists'.
NameError: name 'sum_of_lists' is not defined


As before, the notebook sends the result to the pager, but it looks something like this:

```
Timer unit: 1e-06 s

Total time: 0.009382 s
File: <ipython-input-19-fa2be176cc3e>
Function: sum_of_lists at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def sum_of_lists(N):
     2         1            2      2.0      0.0      total = 0
     3         6            8      1.3      0.1      for i in range(5):
     4         5         9001   1800.2     95.9          L = [j ^ (j >> i) for j in range(N)]
     5         5          371     74.2      4.0          total += sum(L)
     6         1            0      0.0      0.0      return total
```

The information at the top gives us the key to reading the results: the time is reported in microseconds and we can see where the program is spending the most time.
At this point, we may be able to use this information to modify aspects of the script and make it perform better for our desired use case.

For more information on ``%lprun``, as well as its available options, use the IPython help functionality (i.e., type ``%lprun?`` at the IPython prompt). For pruning line by line in classes see [thread](https://stackoverflow.com/questions/49136737/how-profiling-class-method-using-ipython-lprun-magic-function) and for pruning in functions see the following example:

In [3]:
%%capture
import pandas as pd
import numpy as np
import wandb
    
def initwand():
        wandb.init(project="sklearn-naivebayes-tweets")
        
def plot(nb, X_test, y_test, y_probas, y_pred):
        wandb.sklearn.plot_learning_curve(nb, X_test, y_test)
        wandb.sklearn.plot_roc(y_test, y_probas)
        wandb.sklearn.plot_confusion_matrix(y_test, y_pred, labels=nb.classes_)
        wandb.sklearn.plot_precision_recall(y_test, y_probas)


def tweets():
        
    initwand()

    # Get a pandas DataFrame object of all the data in the csv file:
    df = pd.read_csv('../data/tweets.csv')

    # Get pandas Series object of the "tweet text" column:
    text = df['tweet_text']

    # Get pandas Series object of the "emotion" column:
    target = df['is_there_an_emotion_directed_at_a_brand_or_product']

    # Remove the blank rows from the series:
    target = target[pd.notnull(text)]
    text = text[pd.notnull(text)]

    # Perform feature extraction:
    from sklearn.feature_extraction.text import CountVectorizer
    count_vect = CountVectorizer()
    count_vect.fit(text)
    counts = count_vect.transform(text)


    counts_train = counts[:6000]
    target_train = target[:6000]
    counts_test = counts[6000:]
    target_test = target[6000:]


    # Train with this data with a Naive Bayes classifier:
    from sklearn.naive_bayes import MultinomialNB

    nb = MultinomialNB()
    nb.fit(counts, target)


    X_test = counts_test
    y_test = target_test
    y_probas = nb.predict_proba(X_test)
    y_pred = nb.predict(X_test)

    print("y", y_probas.shape)
    
    plot(nb, X_test, y_test, y_probas, y_pred)

In [4]:
%%capture
# line by line profiler of initwand
%lprun -f plot tweets()

Output for def plot():

```
Timer unit: 1e-06 s

Total time: 32.4217 s
File: <ipython-input-3-35107c23cf15>
Function: plot at line 8

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     8                                           def plot(nb, X_test, y_test, y_probas, y_pred):
     9         1   32126962.0 32126962.0     99.1          wandb.sklearn.plot_learning_curve(nb, X_test, y_test)
    10         1     220273.0 220273.0      0.7          wandb.sklearn.plot_roc(y_test, y_probas)
    11         1      41053.0  41053.0      0.1          wandb.sklearn.plot_confusion_matrix(y_test, y_pred, labels=nb.classes_)
    12         1      33380.0  33380.0      0.1          wandb.sklearn.plot_precision_recall(y_test, y_probas)
    
```

## Profiling Memory Use: ``%memit`` and ``%mprun``

Another aspect of profiling is the amount of memory an operation uses.
This can be evaluated with another IPython extension, the ``memory_profiler``.
As with the ``line_profiler``, we start by ``pip``-installing the extension:

```
$ pip install memory_profiler
```

Then we can use IPython to load the extension:

In [12]:
%load_ext memory_profiler

The memory profiler extension contains two useful magic functions: the ``%memit`` magic (which offers a memory-measuring equivalent of ``%timeit``) and the ``%mprun`` function (which offers a memory-measuring equivalent of ``%lprun``).
The ``%memit`` function can be used rather simply:

In [13]:
%memit sum_of_lists(1000000)

peak memory: 100.08 MiB, increment: 61.36 MiB


We see that this function uses about 100 MB of memory.

For a line-by-line description of memory use, we can use the ``%mprun`` magic.
Unfortunately, this magic works only for functions defined in separate modules rather than the notebook itself, so we'll start by using the ``%%file`` magic to create a simple module called ``mprun_demo.py``, which contains our ``sum_of_lists`` function, with one addition that will make our memory profiling results more clear:

In [14]:
%%file mprun_demo.py
def sum_of_lists(N):
    total = 0
    for i in range(5):
        L = [j ^ (j >> i) for j in range(N)]
        total += sum(L)
        del L # remove reference to L
    return total

Overwriting mprun_demo.py


We can now import the new version of this function and run the memory line profiler:

In [15]:
from mprun_demo import sum_of_lists
%mprun -f sum_of_lists sum_of_lists(1000000)

The result, printed to the pager, gives us a summary of the memory use of the function, and looks something like this:
```
Filename: ./mprun_demo.py

Line #    Mem usage    Increment   Line Contents
================================================
     4     71.9 MiB      0.0 MiB           L = [j ^ (j >> i) for j in range(N)]


Filename: ./mprun_demo.py

Line #    Mem usage    Increment   Line Contents
================================================
     1     39.0 MiB      0.0 MiB   def sum_of_lists(N):
     2     39.0 MiB      0.0 MiB       total = 0
     3     46.5 MiB      7.5 MiB       for i in range(5):
     4     71.9 MiB     25.4 MiB           L = [j ^ (j >> i) for j in range(N)]
     5     71.9 MiB      0.0 MiB           total += sum(L)
     6     46.5 MiB    -25.4 MiB           del L # remove reference to L
     7     39.1 MiB     -7.4 MiB       return total
```
Here the ``Increment`` column tells us how much each line affects the total memory budget: observe that when we create and delete the list ``L``, we are adding about 25 MB of memory usage.
This is on top of the background memory usage from the Python interpreter itself.

For more information on ``%memit`` and ``%mprun``, as well as their available options, use the IPython help functionality (i.e., type ``%memit?`` at the IPython prompt).

### Reference:

https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/01.07-Timing-and-Profiling.ipynb
